https://www.youtube.com/watch?v=bhYulVzYRng


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pdb
import datetime as dt
import re
import time

In [2]:
def get_links_LB(date=dt.date.today().strftime("%d/%m/%Y"),
                 chrome_path=r"C:\Users\johnn\chromedriver\chromedriver.exe",
                 url = "https://www.ladbrokes.com.au/sports/basketball/69460051-basketball-usa-nba/"):
    '''
    Goes through ladbroke's NBA games page and returns a list of links to all open markets.
    '''
    driver = webdriver.Chrome(chrome_path)
    driver.get(url)

    boxes = driver.find_elements_by_class_name("fullbox")
    
    # Filter out promotional markets
    games = [box for box in boxes if 'BASKETBALL USA' in box.text]
    links = []

    
    for game in games:
        # filter out games not in today
        table = game.find_element_by_tag_name('table')
        if date not in table.text:
            # Not today
            continue

        elif 'Phone Betting Only' in table.text:
            # Today but already started
            continue
        
        try:
            link = game.find_element_by_tag_name("a")
            links.append(link.get_attribute('href'))
        except:
            None

    driver.close()
    
    return links

In [3]:
links = get_links_LB()

In [4]:
for link in links:
    print(link)

https://www.ladbrokes.com.au/sports/basketball/70337588-basketball-usa-nba/70337588-indiana-pacers-v-phoenix-suns/
https://www.ladbrokes.com.au/sports/basketball/70337588-basketball-usa-nba/70337593-philadelphia-76ers-v-minnesota-timberwolves/
https://www.ladbrokes.com.au/sports/basketball/70337588-basketball-usa-nba/70337613-atlanta-hawks-v-oklahoma-city-thunder/
https://www.ladbrokes.com.au/sports/basketball/70337588-basketball-usa-nba/70337624-milwaukee-bucks-v-miami-heat/
https://www.ladbrokes.com.au/sports/basketball/70337588-basketball-usa-nba/70337638-denver-nuggets-v-golden-state-warriors/
https://www.ladbrokes.com.au/sports/basketball/70337588-basketball-usa-nba/70353337-los-angeles-lakers-v-chicago-bulls/


In [ ]:
def get_player_markets_LB(link,
                         chrome_path=r"C:\Users\johnn\chromedriver\chromedriver.exe"):
    '''
    Returns a list of strings parsed from ladbrokes player markets.
    '''
    driver = webdriver.Chrome(chrome_path)
    driver.get(link)

    # markets = driver.find_elements_by_class_name('additional-market-description')
    markets = driver.find_elements_by_class_name('additional-market')
    markets_of_interest = [
        'Total Points',
        'Total Rebounds',
        'Total Assists'
    ]

    main_markets = [market for market in markets if re.search(r'Total (Points|Rebounds|Assists)$', market.text)]

    # Scroll to top so that the first box isn't hidden by ladbrokes's ribbon
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    time.sleep(1) # Wait for page to finish scrolling

    # Click on all markets of interest so that their odds appear and record what the odds are
    market_list = []
    prev_market_text = '' # There seems to be triples of every market for some reason
    for market in main_markets:
        market.click()
        time.sleep(1)
        market_text = market.text

        if market_text == prev_market_text:
            # Checks if 
            continue
        else:
            market_list.append(market_text)
            prev_market_text = market_text

    driver.close()
    return market_list

In [ ]:
link = links[-1]
market_list = get_player_markets_LB(link)

In [ ]:
market_list

In [ ]:
odds_str = market_list[0]
odds_str

In [ ]:
re_name = r'^(.*) Total '
re.search(re_name, odds_str).group(1)

In [ ]:
re_stat = r' Total (.+)\n'
re.search(re_stat, odds_str).group(1)

In [ ]:
re_baseline = r'Over \((\d+\.5)\)'
re.search(re_baseline, odds_str).group(1)

In [ ]:
re_odds_over = r'\n(\d+\.\d+)\n'
re.search(re_odds_over, odds_str).group(1)

In [ ]:
re_odds_under = r'\n(\d+\.\d+)$'
re.search(re_odds_under, odds_str).group(1)

In [ ]:
class PlayerMarket():
    def __init__(self, exchange, player, stat, baseline, odds_over, odds_under):
        self.exchange = exchange
        self.player = player
        self.stat = stat
        self.baseline = baseline
        self.odds_over = odds_over
        self.odds_under = odds_under
    
    def get_exchange(self):
        return self.exchange
    
    def get_player(self):
        return self.player
    
    def get_stat(self):
        return self.stat
    
    def get_baseline(self):
        return self.baseline
    
    def get_odds_over(self):
        return self.odds_over
    
    def get_odds_under(self):
        return self.odds_under

    def __repr__(self):
        string_repr = '{} - {}\nOver {:^5}Under {:^4}\n{:^10}{:^10}'.format(
            self.get_player(), self.get_stat(),
            self.get_baseline(), self.get_baseline(),
            self.get_odds_over(), self.get_odds_under())
        return string_repr

    def __string__(self):
        return self.__repr__()

In [ ]:
class PlayerMarketLadbrokes(PlayerMarket):
    def __init__(self, odds_str):
        '''
        Parses the string output scraped from sportsbet website and creates a PlayerMarket instance
        String of format:
        'Reggie Jackson Total Assists\nOver (4.5)\n1.87\nUnder (4.5)\n1.87'
        '''
        # Converts all names to format I have in NBA_Stats
        player_name_dict = {
            }
        # Converts all stat headings to format I have in NBA_Stats
        stat_dict = {
            'Points':'PTS',
            'Rebounds':'REB',
            'Assists':'AST'
            }
        
        # Name
        re_name = r'^(.*) Total '
        name = re.search(re_name, odds_str).group(1)
        
        name = re.sub(r' Jr(\s|$)', ' Jr.', name) # Replace Jr with Jr.
        if name in player_name_dict.keys():
            # Replace ladbrokes' name with nba_stats
            player = player_name_dict[name]
        else:
            player = name

        # Stat
        re_stat = r' Total (.+)\n'
        stat = re.search(re_stat, odds_str).group(1)
        stat_NBA = stat_dict[stat]

        # Baseline
        re_baseline = r'Over \((\d+\.5)\)'
        baseline = re.search(re_baseline, odds_str).group(1)
        baseline_num = float(baseline)

        # Odds over
        re_odds_over = r'\n(\d+\.\d+)\n'
        odds_over = re.search(re_odds_over, odds_str).group(1)
        odds_over_num = float(odds_over)
        
        # Odds under
        re_odds_under = r'\n(\d+\.\d+)$'
        odds_under = re.search(re_odds_under, odds_str).group(1)
        odds_under_num = float(odds_under)
        
        PlayerMarket.__init__(self, 'sportsbet', 
                             player, stat_NBA,
                             baseline_num, odds_over_num, 
                              odds_under_num)
        return None

In [ ]:
test_player_market_class = PlayerMarketLadbrokes(odds_str)
print(test_player_market_class)